In [1]:
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ptfrwrd","key":"4a2f5a982ed53062345976efa42bd4c8"}'}

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
! kaggle competitions download -c house-prices-advanced-regression-techniques

  0% 0.00/13.1k [00:00<?, ?B/s]
100% 13.1k/13.1k [00:00<00:00, 24.8MB/s]
  0% 0.00/31.2k [00:00<?, ?B/s]
100% 31.2k/31.2k [00:00<00:00, 32.7MB/s]
  0% 0.00/441k [00:00<?, ?B/s]
100% 441k/441k [00:00<00:00, 62.4MB/s]
  0% 0.00/450k [00:00<?, ?B/s]
100% 450k/450k [00:00<00:00, 64.0MB/s]


# Explore the data:

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import warnings
warnings.simplefilter(action='ignore')

# use this code to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [27]:
cmap = sns.diverging_palette(220, 10, as_cmap=True)

In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train_test = pd.concat((train, test), sort=False).reset_index(drop=True)
train_test.drop(['Id'], axis=1, inplace=True)

train_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


In [7]:
print('Number of categorial types: ', len(train.select_dtypes(include='object').columns))
print('Categorial types: ', train.select_dtypes(include='object').columns)

Number of categorial types:  43
Categorial types:  Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')


In [10]:
!pip install category_encoders

     |████████████████████████████████| 81kB 3.6MB/s 


In [36]:
# see different categorical encoders

from category_encoders.ordinal import OrdinalEncoder
from category_encoders.woe import WOEEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.sum_coding import SumEncoder
from category_encoders.m_estimate import MEstimateEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder
from category_encoders.helmert import HelmertEncoder
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.james_stein import JamesSteinEncoder
from category_encoders.one_hot import OneHotEncoder

**We will use different types encoders only for one categorical column, because it is well demonstrate work and we have not any problems with RAM.**


# 1 - Ordinal Encoder

One of the most popular encoders is Ordinal Encoder (OE). In OE, each unique category value is assigned an integer value. For example, 'Inside' in the LotConfig column means 1, 'FR2' means 2 and etc. It also called integer encoding and it is easy to reverse.

It used, if an ordinal encoding may be enough for variables. The integer values have a natural ordered relationship between each other and machine learning algorithms may be able to understand and harness this relationship.

This approach has a serious disadvantage: for categorical variables, it imposes an ordinal relationship where no such relationship may exist. This can cause problems and a one-hot encoding may be used instead.


In [55]:
%%time
encoder = OrdinalEncoder()
ordinal_encoder_example  = encoder.fit_transform(train_test['LotConfig'])

CPU times: user 8.68 ms, sys: 0 ns, total: 8.68 ms
Wall time: 7.77 ms


In [58]:
# see  data
ordinal_encoder_example['Original_data'] = train_test['LotConfig']
ordinal_encoder_example = ordinal_encoder_example.rename(columns={'LotConfig': 'Ordinal_data'})
ordinal_encoder_example.head()

,Ordinal_data,Original_data
0,1,Inside
1,2,FR2
2,1,Inside
3,3,Corner
4,2,FR2


# 2 - WOE Encoder

Weight Of Evidence is a usually used target-based encoder in credit scoring.

It is a measure of the “strength” of a grouping for separating good and bad risk (default). It is calculated from the basic odds ratio:

$a = Distribution of Good Credit Outcomes$

$b = Distribution of Bad Credit Outcomes$

$WoE = ln(a / b)$

But if we use formulas as is, it might lead to target leakage and overfit. To avoid that, regularization parameter $a$ is induced and WoE is calculated in the following way:

$ nominator = (n^+ + a ) / (y^+ + 2*a) $

$ denominator = (n - n^+ + a) / (y - y^+ + 2*a) $

$ x^k = ln(nominator/denominator) $


**But, on this dataset perfom WOE Encoder is difficult, becase it needs a binary target value.**

# 3 - Target Encoder

Target Encoding is the most popular type of encoding in Kaggle competitions. It takes information about the target to encode categories, which makes it extremely powerful. The encoded category values are calculated according to the following formulas:

$ s = 1 / (1 + exp(-(n-mdl)/a)) $

$ x^k = prior * (1 - s) + s * n^+ / n $

- $mdl$ - min data (samples) in leaf,
- $a$ - smoothing parameter, which representing the power of regularization.

Recommended values for them are in the range (1, 100). New values of category and values with just a single appearance in the training dataset are replaced with the prior ones.

However, it has a huge disadvantage — target leakage: it uses information about the target. Because of the target leakage, the model overfits the training data which results in unreliable validation and lower test scores.

But you can reduce the effect of target leakage by increase regularization, add random noise to the representation of the category in the training dataset (some sort of augmentation), or use Double Validation.

In [41]:
%%time
TE_encoder = TargetEncoder()
train_te = TE_encoder.fit_transform(train['LotShape'], train['SalePrice'])
test_te = TE_encoder.transform(test['LotShape'])

CPU times: user 39.9 ms, sys: 11 µs, total: 39.9 ms
Wall time: 39.7 ms


In [44]:
# see for train data
target_encoder_example = train_te.rename(columns={'LotShape': 'Target_encoder_data'})
target_encoder_example['Original_data'] = train['LotShape']
target_encoder_example.head()

,Target_encoder_data,Original_data
0,164754.818378,Reg
1,164754.818378,Reg
2,206101.665289,IR1
3,206101.665289,IR1
4,206101.665289,IR1


# 4 - Sum Encoder

It also called Deviation Encoding or Effect Encoding.

Sum Encoder compares the mean of the target variable for a given level of a categorical column to the overall mean of the target. Sum Encoder is commonly used in Linear Regression (LR) types of models. The model with Sum Encoder the intercept represents the grand mean (across all conditions) and the coefficients can be interpreted directly as the main effects.

In [45]:
%%time
SE_encoder = SumEncoder('GarageType')
train_se = SE_encoder.fit_transform(train['GarageType'], train['SalePrice'])
test_se = SE_encoder.transform(test['GarageType'])

CPU times: user 28.6 ms, sys: 1.93 ms, total: 30.6 ms
Wall time: 34.3 ms


In [46]:
# see for train data
sum_encoder_example = train_se.rename(columns={'GarageType': 'Sum_encoder_data'})
sum_encoder_example['Original_data'] = train['GarageType']
sum_encoder_example.head()

,intercept,GarageType_0,GarageType_1,GarageType_2,GarageType_3,GarageType_4,GarageType_5,Original_data
0,1,1.0,0.0,0.0,0.0,0.0,0.0,Attchd
1,1,1.0,0.0,0.0,0.0,0.0,0.0,Attchd
2,1,1.0,0.0,0.0,0.0,0.0,0.0,Attchd
3,1,0.0,1.0,0.0,0.0,0.0,0.0,Detchd
4,1,1.0,0.0,0.0,0.0,0.0,0.0,Attchd


# 5 - M-Estimate Encoder

M-probability estimate of likelihood.

M-Estimate Encoder is a simplified version of Target Encoder. It has only one hyperparameter — m, which represents the power of regularization. The higher value of m results into stronger shrinking. Recommended values for m is in the range of 1 to 100.



In [48]:
%%time
MEE_encoder = MEstimateEncoder()
train_mee = MEE_encoder.fit_transform(train['KitchenQual'], train['SalePrice'])
test_mee = MEE_encoder.transform(train_test['KitchenQual'])

CPU times: user 30.9 ms, sys: 2.07 ms, total: 33 ms
Wall time: 39 ms


In [50]:
# see for train data
me_encoder_example = train_mee.rename(columns={'KitchenQual': 'ME_encoder_data'})
me_encoder_example['Original_data'] = train['KitchenQual']
me_encoder_example.head()

,ME_encoder_data,Original_data
0,212062.881083,Gd
1,140018.161951,TA
2,212062.881083,Gd
3,212062.881083,Gd
4,212062.881083,Gd


# 6 - Leave One Out Encoder

Leave-one-out Encoding (LOO or LOOE) is another example of target-based encoders. The name of the method clearly speaks for itself: we calculate the mean target of category k for observation j if observation j is removed from the dataset:

$x_i^k = (sum_{j\not=i} y_i * (x_j ==k)) - y_i / (sum_{j\not=i}x_j == k )$

While encoding the test dataset, a category is replaced with the mean target of the category k in the train dataset:

$x^k = sum ( y_i * (x_j ==k)) / sum (x_j == k) $

One of the disadvantages of LOO, just like with all other target-based encoders, is target leakage (the similar problem we have with Target encoder). But when it comes to LOO, this problem gets really dramatic, as far as we may perfectly classify the training dataset by making a single split: the optimal threshold for category k could be calculated with the following formula:

$t^k = sum ( y_i * (x_j ==k)) - 0.5 / sum (x_j == k) $

Another problem with LOO is a shift between values in the train and the test samples. It has a high influence if we work with tree-based models.

If we go deeper, the encoding algorithm is slightly different between the training and test data sets. For the training data set, the record under consideration is left out, hence the name Leave One Out. The encoding is as follows for a certain value of a certain categorical variable. For the validation data or prediction data set, the definition is slightly different. We don’t need to leave the current record out and we don’t need the randomness factor. 

In [51]:
%%time
LOOE_encoder = LeaveOneOutEncoder()
train_looe = LOOE_encoder.fit_transform(train['GarageFinish'], train['SalePrice'])
test_looe = LOOE_encoder.transform(test['GarageFinish'])

CPU times: user 114 ms, sys: 8 µs, total: 114 ms
Wall time: 111 ms


In [52]:
# see for train data
loo_encoder_example = train_looe.rename(columns={'GarageFinish': 'LOO_encoder_data'})
loo_encoder_example['Original_data'] = train['GarageFinish']
loo_encoder_example.head()

,LOO_encoder_data,Original_data
0,202053.593824,RFn
1,202117.726841,RFn
2,202017.964371,RFn
3,142159.993377,Unf
4,201955.019002,RFn


# 7 - Helmert Encoder

Helmert coding is a also commonly used type of categorical encoding for regression. 

It compares each level of a categorical variable to the mean of the subsequent levels.

This type of encoding can be useful in certain situations where levels of the categorical variable are ordered, say, from lowest to highest, or from smallest to largest.

In [53]:
%%time
HE_encoder = HelmertEncoder('Foundation')
train_he = HE_encoder.fit_transform(train['Foundation'], train['SalePrice'])
test_he = HE_encoder.transform(test['Foundation'])

CPU times: user 28.2 ms, sys: 0 ns, total: 28.2 ms
Wall time: 28.9 ms


In [54]:
# see for train data
he_encoder_example = train_he.rename(columns={'Foundation': 'HE_encoder_data'})
he_encoder_example['Original_data'] = train['Foundation']
he_encoder_example.head()

,intercept,Foundation_0,Foundation_1,Foundation_2,Foundation_3,Foundation_4,Original_data
0,1,-1.0,-1.0,-1.0,-1.0,-1.0,PConc
1,1,1.0,-1.0,-1.0,-1.0,-1.0,CBlock
2,1,-1.0,-1.0,-1.0,-1.0,-1.0,PConc
3,1,0.0,2.0,-1.0,-1.0,-1.0,BrkTil
4,1,-1.0,-1.0,-1.0,-1.0,-1.0,PConc


# 8 - CatBoost Encoder

Catboost is a recently created target-based categorical encoder. It is intended to overcome target leakage problems inherent in LOO. In order to do that, the authors of Catboost introduced the idea of “time”: the order of observations in the dataset. Clearly, the values of the target statistic for each example rely only on the observed history. To calculate the statistic for observation j in train dataset, we may use only observations, which are collected before observation j, i.e. i≤j:

$x_i^k = (sum_{j=0}^{j≤i} y_i * (x_j ==k)) - y_i + prior) / (sum_{j=0}^{j≤i}x_j == k )$


To prevent overfitting, the process of target encoding for train dataset is repeated several times on shuffled versions of the dataset and results are averaged. Encoded values of the test data are calculated the same way as in LOO Encoder:

$x^k = sum ( y_i * (x_j ==k)) +prior / sum (x_j == k) $




In [60]:
%%time
CB_encoder = CatBoostEncoder()
train_cb = CB_encoder.fit_transform(train['Neighborhood'], train['SalePrice'])
test_cb = CB_encoder.transform(test['Neighborhood'])

CPU times: user 147 ms, sys: 572 µs, total: 147 ms
Wall time: 149 ms


In [61]:
# see for train data
cb_encoder_example = train_cb.rename(columns={'Neighborhood': 'CB_encoder_data'})
cb_encoder_example['Original_data'] = train['Neighborhood']
cb_encoder_example.head()

,CB_encoder_data,Original_data
0,180921.195890,CollgCr
1,180921.195890,Veenker
2,194710.597945,CollgCr
3,180921.195890,Crawfor
4,180921.195890,NoRidge


# 9 - James-Stein Encoder

James-Stein Encoder is a target-based encoder.

The idea behind James-Stein Encoder is simple. Estimation of the mean target for category k could be calculated according to the following formula:

$ x^k = (1-B) * n^+/ n + B*y^+/y$

Encoding is aimed to improve the estimation of the category’s mean target (first member of the amount) by shrinking them towards a more central average (second member of the amount). The only hyperparameter in the formula is $B$ — the power of shrinking. It could be understood as the power of regularization, i.e. the bigger values of $B$ will result in the bigger weight of global mean (underfit), while the lower values of $B$ are, the bigger weight of condition mean (overfit).
One way to select $B$ is to tune it like a hyperparameter via cross-validation, but Charles Stein came up with another solution to the problem:

$B = Var[y^k] / (Va[y^k]+Var[y])$

In [62]:
%%time
JS_encoder = JamesSteinEncoder()
train_js = JS_encoder.fit_transform(train['SaleCondition'], train['SalePrice'])
test_js = JS_encoder.transform(test['SaleCondition'])

CPU times: user 35.3 ms, sys: 1.19 ms, total: 36.5 ms
Wall time: 33.6 ms


In [63]:
# see for train data
js_encoder_example = train_js.rename(columns={'SaleCondition': 'JS_encoder_data'})
js_encoder_example['Original_data'] = train['SaleCondition']
js_encoder_example.head()

,JS_encoder_data,Original_data
0,176695.044855,Normal
1,176695.044855,Normal
2,176695.044855,Normal
3,157271.404726,Abnorml
4,176695.044855,Normal


# 10 - OneHotEncoder

The One Hot Encoding is another simple way to work with categorical columns. It takes a categorical column that has been Label Encoded and then splits the column into multiple columns. The numbers are replaced by 1s and 0s depending on which column has what value.
OHE expands the size of your dataset, which makes it memory-inefficient encoder. There are several strategies to overcome the memory problem with OHE, one of which is working with sparse not dense data representation.

In [67]:
# with  category_encoders, but the most common approach with pandas dummy.
%%time
OHE_encoder = OneHotEncoder('RoofStyle')
train_ohe = OHE_encoder.fit_transform(train['RoofStyle'])
test_ohe = OHE_encoder.transform(test['RoofStyle'])

CPU times: user 29.8 ms, sys: 2 µs, total: 29.8 ms
Wall time: 28.5 ms


In [68]:
# see for train data
oh_encoder_example = train_ohe.rename(columns={'RoofStyle': 'OH_encoder_data'})
oh_encoder_example['Original_data'] = train['RoofStyle']
oh_encoder_example.head()

,RoofStyle_1,RoofStyle_2,RoofStyle_3,RoofStyle_4,RoofStyle_5,RoofStyle_6,Original_data
0,1,0,0,0,0,0,Gable
1,1,0,0,0,0,0,Gable
2,1,0,0,0,0,0,Gable
3,1,0,0,0,0,0,Gable
4,1,0,0,0,0,0,Gable
